In [1]:
# If running locally
import sys
sys.path.insert(0, "../")

In [ ]:
# If using Colab
!git clone https://github.com/lollcat/FAB-JAX
!pip install git+https://github.com/deepmind/dm-haiku
!pip install git+git://github.com/deepmind/distrax.git
!pip install git+git://github.com/deepmind/optax.git
import os
os.chdir("FAB-JAX")

In [2]:
from fab.learnt_distributions.real_nvp import make_realnvp_dist_funcs
from fab.target_distributions.many_well import ManyWellEnergy
from fab.agent.fab_agent import AgentFAB
from fab.utils.plotting import plot_history, plot_marginal_pair, plot_contours_2D
import matplotlib.pyplot as plt
import optax
import jax
import jax.numpy as jnp

2022-03-06 17:31:22.686573: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [11]:
dim = 8
flow_num_layers = 10
mlp_hidden_size_per_x_dim = 5
layer_norm = False
act_norm = True

In [12]:
real_nvp_flo = make_realnvp_dist_funcs(dim, flow_num_layers,
                                       mlp_hidden_size_per_x_dim=mlp_hidden_size_per_x_dim, 
                                       use_exp=True, layer_norm=layer_norm, act_norm=act_norm)
target_log_prob = ManyWellEnergy(dim=dim).log_prob
batch_size = 124
n_iter = int(2e4)
lr = 1e-4
n_intermediate_distributions: int = 2
AIS_kwargs = {"additional_transition_operator_kwargs": {"step_tuning_method": "p_accept"}}

In [13]:
fab_agent = AgentFAB(learnt_distribution=real_nvp_flo,
                     target_log_prob=target_log_prob,
                     batch_size=batch_size, n_iter=n_iter,
                     n_intermediate_distributions=n_intermediate_distributions,
                     AIS_kwargs=AIS_kwargs,
                     optimizer=optax.adam(lr))

using linear spacing as there is 2intermediate distribution


In [ ]:
fab_agent.run()

ESS: 0.1759492732660775:  21%|█████████████████████████████████                                                                                                                           | 4243/20000 [02:19<04:57, 52.92it/s]

In [ ]:
plot_history(fab_agent.history)
plt.show()

In [ ]:
def plot(fab_agent, n_samples = batch_size, dim=dim, key=jax.random.PRNGKey(0)):
    plotting_bounds = 3
    target = ManyWellEnergy(dim=dim)
    n_rows = dim // 2
    fig, axs = plt.subplots(dim // 2, 2,  sharex=True, sharey=True, figsize=(10, n_rows*3))

    
    samples_flow = fab_agent.learnt_distribution.sample.apply(fab_agent.learnt_distribution_params, key, (batch_size,))
    samples_ais = fab_agent.annealed_importance_sampler.run(key, fab_agent.learnt_distribution_params)[0]

    for i in range(n_rows):
      plot_contours_2D(target.log_prob_2D, bound=plotting_bounds, ax=axs[i, 0])
      plot_contours_2D(target.log_prob_2D, bound=plotting_bounds, ax=axs[i, 1])

      # plot flow samples
      plot_marginal_pair(samples_flow, ax=axs[i, 0], bounds=(-plotting_bounds, plotting_bounds), marginal_dims=(i*2,i*2+1))
      axs[i, 0].set_xlabel(f"dim {i*2}")
      axs[i, 0].set_ylabel(f"dim {i*2 + 1}")
      


      # plot ais samples
      plot_marginal_pair(samples_ais, ax=axs[i, 1], bounds=(-plotting_bounds, plotting_bounds), marginal_dims=(i*2,i*2+1))
      axs[i, 1].set_xlabel(f"dim {i*2}")
      axs[i, 1].set_ylabel(f"dim {i*2+1}")
      plt.tight_layout()
    axs[0, 1].set_title("ais samples")  
    axs[0, 0].set_title("flow samples")
    plt.show()

In [ ]:
plot(fab_agent)